In [23]:
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn import linear_model


In [2]:
data = sns.load_dataset("tips")
data.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## We will train an AI to predict what the tip will be based off of the bill

In [3]:
features = data[["total_bill"]]
tip = data["tip"]

from sklearn import linear_model
f = linear_model.LinearRegression(fit_intercept = False)
f.fit(features, tip)

LinearRegression(fit_intercept=False)

In [5]:
f.predict([[74.12]])

c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([10.65340808])

In [6]:
#add predictions to dataframe
data["prediction"] = f.predict(data[["total_bill"]])

In [7]:
data.head(5)

,total_bill,tip,sex,smoker,day,time,size,prediction
0,16.99,1.01,Female,No,Sun,Dinner,2,2.442005
1,10.34,1.66,Male,No,Sun,Dinner,3,1.486188
2,21.01,3.50,Male,No,Sun,Dinner,3,3.019807
3,23.68,3.31,Male,No,Sun,Dinner,2,3.403571
4,24.59,3.61,Female,No,Sun,Dinner,4,3.534367


In [8]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['total_bill'], y = data['tip'], mode = "markers", name = "actual"))
fig.add_trace(go.Scatter(x=data['total_bill'], y = data['prediction'], mode = "lines", name = "predicted"))
fig.update_layout(font_size = 20)

In [9]:
f.coef_

array([0.1437319])

In [10]:
f.intercept_

0.0

In [11]:
# Now we're going to create a model where the intercept is not forced to be 0

f_w_int = linear_model.LinearRegression(fit_intercept = True)
f_w_int.fit(features, tip)

LinearRegression()

In [12]:
f_w_int.predict([[100]])

c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



array([11.42272135])

In [13]:
f.predict([[100]])

c:\Users\sspillane\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



array([14.37318953])

In [14]:
f_w_int.coef_

array([0.10502452])

In [15]:
f_w_int.intercept_

0.9202696135546731

In [16]:
#get coefficient and intercept for new model
# Plot both of them together

In [17]:
data['prediction_with_intercept'] = f_w_int.predict(data[["total_bill"]])
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['total_bill'], y = data['tip'], 
                         mode = "markers", name = "actual"))
fig.add_trace(go.Scatter(x=data['total_bill'], y = data['prediction'], 
                         mode = "lines", name = "prediction (b = 0)"))
fig.add_trace(go.Scatter(x=data['total_bill'], y = data['prediction_with_intercept'], 
                         mode = "lines", name = "predicted (b ≠ 0)"))
fig.update_layout(font_size = 20)

The formulas for the fit lines are then
1. y = 0.1437x
2. y = 0.105x + 0.92

### Using Plotly

In [ ]:
# Nevermind, we aren'd really going to use this much...

#create the plot
fig = px.scatter(data, x = "total_bill", y = "tip", trendline = "ols", trendline_color_override="black")
# fig.update_layout(font_size = 20)

In [ ]:
#get the model parameters

## Loss Functions

### Computing the L2 Loss and MSE

In [27]:
data = sns.load_dataset("tips")
data["prediction"] = f.predict(data[["total_bill"]])
data.head(5)

,total_bill,tip,sex,smoker,day,time,size,prediction
0,16.99,1.01,Female,No,Sun,Dinner,2,2.442005
1,10.34,1.66,Male,No,Sun,Dinner,3,1.486188
2,21.01,3.50,Male,No,Sun,Dinner,3,3.019807
3,23.68,3.31,Male,No,Sun,Dinner,2,3.403571
4,24.59,3.61,Female,No,Sun,Dinner,4,3.534367


In [28]:
# add loss to the dataframe
data["L2_loss"] = (data["prediction"] - data["tip"])**2
data.head(5)

,total_bill,tip,sex,smoker,day,time,size,prediction,L2_loss
0,16.99,1.01,Female,No,Sun,Dinner,2,2.442005,2.050638
1,10.34,1.66,Male,No,Sun,Dinner,3,1.486188,0.030211
2,21.01,3.50,Male,No,Sun,Dinner,3,3.019807,0.230585
3,23.68,3.31,Male,No,Sun,Dinner,2,3.403571,0.008756
4,24.59,3.61,Female,No,Sun,Dinner,4,3.534367,0.005720


In [29]:
#compute the mean
np.mean(data["L2_loss"])

1.1781161154513171

In [30]:
# here's another way...

data["L2_loss"].mean()

1.1781161154513171

In [34]:
## or we can compute MSE directly from the outcome and predictions using mean_squared_error
from sklearn.metrics import mean_squared_error
mean_squared_error(data["tip"], f.predict(data[["total_bill"]]))

1.1781161154513171

### Optimizing the MSE

In [40]:
# Replace theprediction column and L2 loss columns with the corresponding values for 20% tip
# The mean squared error will change, and the SciKit algorithm tries different values to maximize it

data["prediction"] = 0.11 * data["total_bill"]
data["L2_loss"] = (data["prediction"] - data["tip"])**2
data

,total_bill,tip,sex,smoker,day,time,size,prediction,L2_loss
0,16.99,1.01,Female,No,Sun,Dinner,2,1.8689,0.737709
1,10.34,1.66,Male,No,Sun,Dinner,3,1.1374,0.273111
2,21.01,3.50,Male,No,Sun,Dinner,3,2.3111,1.413483
3,23.68,3.31,Male,No,Sun,Dinner,2,2.6048,0.497307
4,24.59,3.61,Female,No,Sun,Dinner,4,2.7049,0.819206
...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,3.1933,7.434893
240,27.18,2.00,Female,Yes,Sat,Dinner,2,2.9898,0.979704
241,22.67,2.00,Male,Yes,Sat,Dinner,2,2.4937,0.243740
242,17.82,1.75,Male,No,Sat,Dinner,2,1.9602,0.044184


In [42]:
mean_squared_error(data["prediction"], data["tip"])

1.7133696722540983

We can compute the MSE for various values of theta.

In [43]:
## error assuming 20% tip
mean_squared_error(data['tip'], data['total_bill']*0.2)

2.667486278688525

In [44]:
## error assuming 10% tip
mean_squared_error(data['tip'], data['total_bill']*0.1)

2.0777683729508194

In [45]:
## error assuming 13% tip
mean_squared_error(data['tip'], data['total_bill']*0.13)

1.2668192650409837

In [46]:
mean_squared_error(data['tip'], data['total_bill']*0.14)

1.18466755852459

In [47]:
# define a function mse_given_theta as a function which takes theta
# and computes RMSE
def mse_given_theta(theta):
    return mean_squared_error(data["tip"], data["total_bill"]*theta)

In [48]:
mse_given_theta(0.145)

1.178872579538934

In [49]:
# compute MSEs for various thetas
thetas = np.linspace(0.1, 0.2, 100)
thetas

array([0.1       , 0.1010101 , 0.1020202 , 0.1030303 , 0.1040404 ,
       0.10505051, 0.10606061, 0.10707071, 0.10808081, 0.10909091,
       0.11010101, 0.11111111, 0.11212121, 0.11313131, 0.11414141,
       0.11515152, 0.11616162, 0.11717172, 0.11818182, 0.11919192,
       0.12020202, 0.12121212, 0.12222222, 0.12323232, 0.12424242,
       0.12525253, 0.12626263, 0.12727273, 0.12828283, 0.12929293,
       0.13030303, 0.13131313, 0.13232323, 0.13333333, 0.13434343,
       0.13535354, 0.13636364, 0.13737374, 0.13838384, 0.13939394,
       0.14040404, 0.14141414, 0.14242424, 0.14343434, 0.14444444,
       0.14545455, 0.14646465, 0.14747475, 0.14848485, 0.14949495,
       0.15050505, 0.15151515, 0.15252525, 0.15353535, 0.15454545,
       0.15555556, 0.15656566, 0.15757576, 0.15858586, 0.15959596,
       0.16060606, 0.16161616, 0.16262626, 0.16363636, 0.16464646,
       0.16565657, 0.16666667, 0.16767677, 0.16868687, 0.16969697,
       0.17070707, 0.17171717, 0.17272727, 0.17373737, 0.17474

In [51]:
mses = [mse_given_theta(theta) for theta in thetas]
mses

[2.0777683729508194,
 2.0366887534058913,
 1.996569059699077,
 1.9574092918303747,
 1.919209449799786,
 1.8819695336073097,
 1.8456895432529465,
 1.8103694787366964,
 1.7760093400585586,
 1.7426091272185338,
 1.7101688402166224,
 1.678688479052823,
 1.6481680437271375,
 1.6186075342395636,
 1.5900069505901033,
 1.5623662927787565,
 1.5356855608055218,
 1.5099647546704,
 1.4852038743733909,
 1.461402919914495,
 1.4385618912937121,
 1.4166807885110417,
 1.3957596115664843,
 1.37579836046004,
 1.3567970351917082,
 1.3387556357614898,
 1.3216741621693837,
 1.3055526144153906,
 1.2903909924995107,
 1.2761892964217436,
 1.262947526182089,
 1.2506656817805475,
 1.2393437632171185,
 1.2289817704918033,
 1.2195797036046003,
 1.2111375625555103,
 1.2036553473445333,
 1.197133057971669,
 1.1915706944369175,
 1.1869682567402793,
 1.1833257448817533,
 1.1806431588613406,
 1.1789204986790405,
 1.1781577643348538,
 1.1783549558287796,
 1.1795120731608182,
 1.18162911633097,
 1.1847060853392344,
 1.18

In [52]:
fig = px.line(x = thetas, y = mses)
fig.update_layout(
    xaxis_title = "θ",
    yaxis_title = "MSE",
    font_size = 20
)
fig.show()

## Minimizing a Function Using Scipy Optimize

In [104]:
import scipy.optimize

In [105]:
def g(x):
    return x**3 + x**2 - 3*x + 2

In [106]:
g(0)

2

In [108]:
g(0.1)

1.7109999999999999

In [113]:
scipy.optimize.minimize(g, x0 = -5)

      fun: array([-1.12131661e+09])
 hess_inv: array([[-0.00031942]])
      jac: array([3237968.])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 236
      nit: 1
     njev: 112
   status: 2
  success: False
        x: array([-1039.24])

In [112]:
scipy.optimize.minimize(mse_given_theta, x0 = 3)

      fun: 1.1781161154513435
 hess_inv: array([[0.0010629]])
      jac: array([1.49011612e-08])
  message: 'Optimization terminated successfully.'
     nfev: 18
      nit: 4
     njev: 9
   status: 0
  success: True
        x: array([0.14373189])

## Multiple Linear Regression

In [157]:
# fit a model on total_bill and size
two_features = data[["total_bill", "size"]]
tip = data["tip"]

f2 = linear_model.LinearRegression(fit_intercept = False)
f2.fit(two_features, tip)

LinearRegression(fit_intercept=False)

In [159]:
#show coefficients
f2.coef_

array([0.1007119 , 0.36209717])

In [160]:
# make a prediction for a table with $10 total bill and 3 people seated
f2.predict([[10, 3]])

array([2.09341054])

In [164]:
data["prediction"] = f.predict(data[["total_bill"]])

In [165]:
data["prediction_2d"] = f2.predict(data[["total_bill", "size"]])
data.head(5)

,total_bill,tip,sex,smoker,day,time,size,prediction,L2_loss,predictions,prediction_2d
0,16.99,1.01,Female,No,Sun,Dinner,2,2.442005,5.702544,2.435290,2.435290
1,10.34,1.66,Male,No,Sun,Dinner,3,1.486188,0.166464,2.127653,2.127653
2,21.01,3.50,Male,No,Sun,Dinner,3,3.019807,0.492804,3.202249,3.202249
3,23.68,3.31,Male,No,Sun,Dinner,2,3.403571,2.033476,3.109052,3.109052
4,24.59,3.61,Female,No,Sun,Dinner,4,3.534367,1.710864,3.924894,3.924894


In [166]:
px.scatter_3d(data, x = "total_bill", y = "size", z = "tip")

In [167]:
# This code is not something we expect you to understand!
# It's just computing predictions for various bills and table sizes
table_bills, table_sizes = np.meshgrid(range(50), range(6))
tip_predictions = ( 0.1007119 * table_bills + 0.3621 * table_sizes)

In [168]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x = data["total_bill"], y = data["size"], z = data["tip"],
                         mode = "markers", name = "actual"))
fig.add_trace(go.Surface(x = table_bills, y = table_sizes, z = tip_predictions, name = "predicted"))
fig.show()

In [169]:
mean_squared_error(data["tip"], f.predict(data[["total_bill"]]))

1.178116115451317

In [170]:
mean_squared_error(data["tip"], f2.predict(data[["total_bill", "size"]]))

1.06482122862577

### Using Non-numeric Features

In [194]:
data = sns.load_dataset("tips")
data.iloc[[193, 90, 25, 26, 190], :]

,total_bill,tip,sex,smoker,day,time,size
193,15.48,2.02,Male,Yes,Thur,Lunch,2
90,28.97,3.00,Male,Yes,Fri,Dinner,2
25,17.81,2.34,Male,No,Sat,Dinner,4
26,13.37,2.00,Male,No,Sat,Dinner,2
190,15.69,1.50,Male,Yes,Sun,Dinner,2


In [196]:
# Let's create a copy of the dataset that only has 3 features in order to keep things simple
three_features = ['total_bill', 'size', 'day']

three_feature_data = pd.DataFrame(data[three_features])
three_feature_data.head(5)

,total_bill,size,day
0,16.99,2,Sun
1,10.34,3,Sun
2,21.01,3,Sun
3,23.68,2,Sun
4,24.59,4,Sun


We can achieve our goal by using the pd.get_dummies function.

In [197]:
# let's create "dummies" that represent whether it is thursday, friday, saturday, or sunday
dummies = pd.get_dummies(three_feature_data['day'])
dummies.iloc[[193, 90, 25, 26, 190], :]

,Thur,Fri,Sat,Sun
193,1,0,0,0
90,0,1,0,0
25,0,0,1,0
26,0,0,1,0
190,0,0,0,1


In [198]:
dummies = pd.get_dummies(three_feature_data['day'])
data_w_dummies = pd.concat([three_feature_data, dummies], axis=1)
data_w_dummies.iloc[[193, 90, 25, 26, 190], :]

,total_bill,size,day,Thur,Fri,Sat,Sun
193,15.48,2,Thur,1,0,0,0
90,28.97,2,Fri,0,1,0,0
25,17.81,4,Sat,0,0,1,0
26,13.37,2,Sat,0,0,1,0
190,15.69,2,Sun,0,0,0,1


As you'll see when you run the cell below, the data frame above will cause the linear regression fitter to crash due to the one non-numeric column.

In [200]:
# The code below will crash since data_w_dummies includes a non-numeric feature
f_with_day = linear_model.LinearRegression(fit_intercept=False)
f_with_day.fit(data_w_dummies, tip)

ValueError: could not convert string to float: 'Sun'

In [202]:
#drop the non-numeric column
del data_w_dummies["day"]
data_w_dummies.head(5)

,total_bill,size,Thur,Fri,Sat,Sun
0,16.99,2,0,0,0,1
1,10.34,3,0,0,0,1
2,21.01,3,0,0,0,1
3,23.68,2,0,0,0,1
4,24.59,4,0,0,0,1


Now that we've dropped the non-numeric data, we can fit our model.

In [203]:
f_with_day = linear_model.LinearRegression(fit_intercept=False)
f_with_day.fit(data_w_dummies, tip)

LinearRegression(fit_intercept=False)

In [204]:
f_with_day.coef_

array([0.09299361, 0.18713231, 0.66829361, 0.74578683, 0.62112858,
       0.73228865])

In [206]:
## Prediction for a party of 3, with a $50 total bill, eating on a thursday
f_with_day.predict([[50, 3, 1, 0, 0, 0]])

array([5.87937107])

In [208]:
## Prediction for a party of 3 with a $50 total bill, eating on a saturday
f_with_day.predict([[50, 3, 0, 0, 0, 1]])

array([5.94336612])

In [209]:
## Let's compute the MSE
mean_squared_error(data['tip'], f_with_day.predict(data_w_dummies))

1.0121298853078433

### Fitting Multiple Non-Numeric Conditions in Plotly

Rather than fitting a higher dimensional (more parameters) model, we can also sometimes simply build visualizations on specific subpopulations.

In [211]:
px.scatter(data, x = "total_bill", y =  "tip", color = "day", trendline = "ols")